In [ ]:
! pwd
! ls -F
from __future__ import absolute_import, division, print_function, unicode_literals
import foo
#from importlib import reload
#foo=reload(foo)
foo.main()
import tensorflow as tf
from tensorflow.python.framework import test_util
from tensorflow.keras.models import Sequential
from tensorflow import keras as K
from tensorflow.keras.layers import Dense, Flatten, Softmax
#import tensorflow_datasets as tfds
print(f'xx tensorflow {tf.__version__}')
import bar as F
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
time,series=F.generateTimeSeriesData()
plt.figure(figsize=(10, 6))
F.plot_series(time,series,title='Original Series')
plt.show()

In [ ]:
# Test your function
time,series=F.generateTimeSeriesData()
print(time.shape)
splitTime=1100
time_train, series_train, time_valid, series_valid = F.train_val_split(time,series,timeStep)

plt.figure(figsize=(10, 6))
F.plot_series(time_train, series_train, title="Training")
plt.show()

plt.figure(figsize=(10, 6))
F.plot_series(time_valid, series_valid, title="Validation")
plt.show()

In [ ]:
dir(F)
F.testComputeMetrics()

In [ ]:
naive_forecast =  series[splitTime-1:-1]
print(f"validation series has shape: {series_valid.shape}\n")
print(f"naive forecast has shape: {naive_forecast.shape}\n")
print(f"comparable with validation series: {series_valid.shape == naive_forecast.shape}")
plt.figure(figsize=(10, 6))
F.plot_series(time_valid, series_valid, label="validation set")
F.plot_series(time_valid, naive_forecast, label="naive forecast",title='Naive Forecast on Validation Set')

In [ ]:
# zoom in
plt.figure(figsize=(10, 6))
F.plot_series(time_valid, series_valid, start=330, end=361, label="validation set")
F.plot_series(time_valid, naive_forecast, start=330, end=361, label="naive forecast",title='Zoom in on Naive Forecast on Validation Set')

In [ ]:
mse, mae = F.compute_metrics(series_valid, naive_forecast)

print(f"mse: {mse:.2f}, mae: {mae:.2f} for naive forecast")
# expected: mse: 19.58, mae: 2.60 for naive forecast